# Try to optimize LT and GEO maps by minimizing E resolution

### Notebook configuration

In [1]:
run_number             = 4734
correction_filename    = f"$IC_DATA/XYmaps/corrections_run{run_number}.h5"
dst_filename           = f"$IC_DATA/Kr/dst_{run_number}_filtered.root.h5"

XY_pitch            = 5.0

Zrange              =     0, 600
XYrange             =  -200, 200
Rrange              =     0, max(XYrange)
Phirange            = -3.14, 3.14

Erange              =    35, 50
Qrange              =   200, 1500
QErange             =     0, Qrange[1] / Erange[1]

S2Erange            =     0, 18e3

Zrange_LT           =    50, 520

XYnbins             =  80
Znbins              = 100
Rnbins              = 100
Enbins              = 200
Qnbins              = 200

S2Enbins            = 100

R_fiducial          = 100
Z_fiducial          = 200

# Plotting style
default_cmap        = "jet"
figure_size         = 16, 12
axis_font_size      = 40
label_font_size     = 30
global_linewidth    = 4
subfit_linewidth    = 4
global_linecolor    = "r"
subfit_linecolor    = "m g b y".split()
subfit_linecolor    = "b m slategrey c b y".split()

### Imports

In [20]:
import os
import time
import collections
import itertools
print("Running on ", time.asctime())

import numpy             as np
import scipy.optimize    as opt
import matplotlib.pyplot as plt

import invisible_cities.database.load_db        as DB
import invisible_cities.core    .core_functions as coref
import invisible_cities.core    .fit_functions  as fitf
import invisible_cities.reco    .corrections    as corrf
import invisible_cities.reco    .dst_functions  as dstf
import invisible_cities.io      .dst_io         as dstio

from invisible_cities.core .stat_functions import poisson_sigma
from invisible_cities.icaro. hst_functions import hist
from invisible_cities.icaro. hst_functions import hist2d
from invisible_cities.icaro. hst_functions import pdf
from invisible_cities.icaro. hst_functions import display_matrix
from invisible_cities.icaro. hst_functions import resolution
from invisible_cities.icaro. hst_functions import gausstext
from invisible_cities.icaro. hst_functions import shift_to_bin_centers
from invisible_cities.icaro. hst_functions import plot_writer
from invisible_cities.icaro. hst_functions import measurement_string

from icaro.core.fit_functions import conditional_labels
from icaro.core.fit_functions import expo_seed

Running on  Thu Dec 20 17:11:37 2018


### Initialization

In [6]:
%matplotlib inline

plt.rcParams["figure.figsize"         ] =    figure_size
plt.rcParams[  "font.size"            ] = axis_font_size
plt.rcParams["figure.max_open_warning"] = 1000

In [7]:
Zbins     = np.linspace(*  Zrange   ,   Znbins + 1)
Ebins     = np.linspace(*  Erange   ,   Enbins + 1)
Qbins     = np.linspace(*  Qrange   ,   Qnbins + 1)
S2Ebins   = np.linspace(*S2Erange   , S2Enbins + 1)

XYbins    = np.linspace(* XYrange   ,  XYnbins + 1)
Rbins     = np.linspace(*  Rrange   ,   Rnbins + 1)

dst_filename           = os.path.expandvars(       dst_filename)
correction_filename    = os.path.expandvars(correction_filename)

In [8]:
Erange              =  35, 50
Erange              =  33.5, 49.5
Ebins     = np.linspace(*  Erange   ,   Enbins + 1)

### Read data

In [9]:
dst = dstf.load_dst(dst_filename, "DST", "Events")
dst = dst[dst.R < max(Rrange)] # soft radial cut to reduce badly reconstructed events
print(len(dst))
dst.head()

906981


,index,event,time,peak,nS2,S1w,S1h,S1e,S1t,S2w,...,S2t,Nsipm,DT,Z,X,Y,R,Phi,Xrms,Yrms
0,0,6,1.507368e+09,0,1,150.0,0.918589,5.302018,260975.0,10.7625,...,654500.0,32,393.525,393.525,110.109160,76.577461,134.119852,0.607678,13.281222,12.380607
1,1,8,1.507368e+09,0,1,125.0,1.663614,7.505072,311675.0,10.4875,...,654500.0,31,342.825,342.825,-84.927306,42.502615,94.969045,2.677578,12.633615,15.235298
2,3,12,1.507368e+09,0,1,225.0,2.741303,17.944131,209450.0,10.8625,...,653500.0,29,444.050,444.050,-34.455078,145.627975,149.648453,1.803121,12.388186,25.229376
3,5,17,1.507368e+09,0,1,200.0,1.953479,9.259772,240275.0,11.4375,...,654500.0,29,414.225,414.225,81.499882,-74.629563,110.507025,-0.741423,25.768544,12.567276
4,6,18,1.507368e+09,0,1,300.0,5.919568,32.076883,269525.0,10.6625,...,653500.0,34,383.975,383.975,15.658394,-195.197920,195.824955,-1.490750,20.252646,26.597081


In [10]:
ELT_correction  = dstf.load_lifetime_xy_corrections(correction_filename,
                                                    group = "XYcorrections",
                                                    node  = "Lifetime")

EGEO_correction = dstf.load_xy_corrections(correction_filename,
                                           group         =  "XYcorrections",
                                           node          = f"GeometryE_{XY_pitch:.1f}mm",
                                           norm_strategy =  "index",
                                           norm_opts     = {"index": (40, 40)})

In [84]:
evt = dst.event.values
T   =(dst.time .values -
      dst.time .values.min()) / 60
S2e = dst.S2e.values
S2t = dst.S2t.values

X   = dst.X    .values
Y   = dst.Y    .values
Z   = dst.DT   .values
R   = dst.R    .values

E   = S2e * ELT_correction(Z, X, Y).value * EGEO_correction(X, Y).value

In [85]:
t, e, eu = fitf.profileX(T, E, 100)
ET_correction = corrf.Correction((t,), e, eu,
                                 norm_strategy =  "max")

E = E * ET_correction(T).value

# Minimization

In [53]:
with tb.open_file(correction_filename) as file: print(file)

/Users/Gonzalo/github/NEXTdata//XYmaps/corrections_run4734.h5 (File) ''
Last modif.: 'Wed Mar 14 02:09:51 2018'
Object Tree: 
/ (RootGroup) ''
/XYcorrections (Group) ''
/XYcorrections/Escale (Table(7200,), shuffle, zlib(4)) 'XY-dependent energy scale'
/XYcorrections/GeometryE_10.0mm (Table(1600,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_2.0mm (Table(40000,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_2.2mm (Table(32400,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_2.5mm (Table(25600,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_2.9mm (Table(19600,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_3.3mm (Table(14400,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_4.0mm (Table(10000,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_5.0mm (Table(6400,), shuffle, zlib(4)) 'XY corrections'
/XYcorrections/GeometryE_6.7mm (Table(3600,), shuffle, zlib(4)) 'XY corrections'
/XYcorrectio

In [112]:
geo   = dstio.load_dst(correction_filename, "XYcorrections", f"GeometryE_5.0mm")
geo_x = np.unique(geo.x.values)
geo_y = np.unique(geo.y.values)
geo0  = geo.factor.values
geou  = geo.uncertainty.values.reshape(80, 80)

lt   = dstio.load_dst(correction_filename, "XYcorrections", f"Lifetime")
lt_x = np.unique(lt.x.values)
lt_y = np.unique(lt.y.values)
lt0  = lt.factor.values
ltu  = lt.uncertainty.values.reshape(60, 60)

In [177]:
def energy_rms(all_coeffs):
    print("call", np.where(all_coeffs != pars0))
    lt_coeffs  = np.reshape(all_coeffs[:3600], (60, 60))
    geo_coeffs = np.reshape(all_coeffs[3600:], (80, 80))
    lt_corr    = corrf.LifetimeXYCorrection(lt_coeffs,
                                            ltu,
                                            lt_x, lt_y)
    geo_corr   = corrf.Correction((geo_x, geo_y),
                                  geo_coeffs,
                                  geou,
                                  norm_strategy =  "index",
                                  norm_opts     = {"index": (40, 40)})
    energies   = S2e * lt_corr(Z, X, Y).value * geo_corr(X, Y).value * ET_correction(T).value
    std        = energies.std()
    print(std)
    return std

In [178]:
pars0 = np.concatenate([lt0, geo0])

In [175]:
energy_rms(pars0)

call False
279.0978171667763


279.0978171667763

In [179]:
opt.minimize(energy_rms, pars0, method="Nelder-Mead", options=dict(maxiter=1, disp=True))

call (array([], dtype=int64),)
279.0978171667763
call (array([0]),)
279.0978171667763
call (array([1]),)
279.0978171667763
call (array([2]),)
279.0978171667763
call (array([3]),)
279.0978171667763
call (array([4]),)
279.0978171667763
call (array([5]),)
279.0978171667763
call (array([6]),)
279.0978171667763
call (array([7]),)
279.0978171667763
call (array([8]),)
279.0978171667763
call (array([9]),)
279.0978171667763
call (array([10]),)
279.0978171667763
call (array([11]),)
279.0978171667763
call (array([12]),)
279.0978171667763
call (array([13]),)
279.0978171667763
call (array([14]),)
279.0978171667763
call (array([15]),)
279.0978171667763
call (array([16]),)
279.0978171667763
call (array([17]),)
279.0978171667763
call (array([18]),)
279.0978171667763
call (array([19]),)
279.0978171667763
call (array([20]),)
279.0978171667763
call (array([21]),)
279.0978171667763
call (array([22]),)
279.0970911727879
call (array([23]),)
279.09793088565675
call (array([24]),)
279.10342703357435
call (arr

279.1051764977391
call (array([212]),)
279.1089582907426
call (array([213]),)
279.1017654656998
call (array([214]),)
279.1061624934154
call (array([215]),)
279.1100452787657
call (array([216]),)
279.10835411524295
call (array([217]),)
279.10302502407217
call (array([218]),)
279.11431403561227
call (array([219]),)
279.10810799803625
call (array([220]),)
279.0965274999862
call (array([221]),)
279.1048007485596
call (array([222]),)
279.09876873378175
call (array([223]),)
279.1020108259698
call (array([224]),)
279.0999402312093
call (array([225]),)
279.0978171667763
call (array([226]),)
279.0978171667763
call (array([227]),)
279.0978171667763
call (array([228]),)
279.0978171667763
call (array([229]),)
279.0978171667763
call (array([230]),)
279.0978171667763
call (array([231]),)
279.0978171667763
call (array([232]),)
279.0978171667763
call (array([233]),)
279.0978171667763
call (array([234]),)
279.0978171667763
call (array([235]),)
279.0978171667763
call (array([236]),)
279.0978171667763
ca

279.0978171667763
call (array([422]),)
279.0978171667763
call (array([423]),)
279.0978171667763
call (array([424]),)
279.0978171667763
call (array([425]),)
279.0978171667763
call (array([426]),)
279.0978171667763
call (array([427]),)
279.0978171667763
call (array([428]),)
279.0978171667763
call (array([429]),)
279.0979008068791
call (array([430]),)
279.09973623833355
call (array([431]),)
279.10080793510394
call (array([432]),)
279.0966883976291
call (array([433]),)
279.0866863963468
call (array([434]),)
279.1025682256387
call (array([435]),)
279.1029915694932
call (array([436]),)
279.1011630258919
call (array([437]),)
279.1048642913436
call (array([438]),)
279.10208911703404
call (array([439]),)
279.1069750578411
call (array([440]),)
279.1026933793432
call (array([441]),)
279.10014444031833
call (array([442]),)
279.1124997803557
call (array([443]),)
279.10267415214605
call (array([444]),)
279.1036792244416
call (array([445]),)
279.10682112448274
call (array([446]),)
279.100816525806
ca

279.1022587717412
call (array([631]),)
279.1029324418496
call (array([632]),)
279.1030200249282
call (array([633]),)
279.1004790870651
call (array([634]),)
279.10016924767774
call (array([635]),)
279.1071140751494
call (array([636]),)
279.10492138157974
call (array([637]),)
279.10354579902537
call (array([638]),)
279.10593551489353
call (array([639]),)
279.0985922150057
call (array([640]),)
279.1102210986082
call (array([641]),)
279.09974310239556
call (array([642]),)
279.101153704665
call (array([643]),)
279.10479345581405
call (array([644]),)
279.10598622141225
call (array([645]),)
279.10694561523553
call (array([646]),)
279.09395577874403
call (array([647]),)
279.10529323417876
call (array([648]),)
279.10370138300203
call (array([649]),)
279.0974038033251
call (array([650]),)
279.1022192028997
call (array([651]),)
279.1062551892623
call (array([652]),)
279.09657346595407
call (array([653]),)
279.09799720308104
call (array([654]),)
279.0978171667763
call (array([655]),)
279.097817166

279.0978171667763
call (array([840]),)
279.0978171667763
call (array([841]),)
279.0978171667763
call (array([842]),)
279.0978171667763
call (array([843]),)
279.0978171667763
call (array([844]),)
279.1008936681844
call (array([845]),)
279.09737446741633
call (array([846]),)
279.1046017763136
call (array([847]),)
279.10530113936386
call (array([848]),)
279.099657751981
call (array([849]),)
279.1069420137366
call (array([850]),)
279.0987331871757
call (array([851]),)
279.103831351538
call (array([852]),)
279.09966711136616
call (array([853]),)
279.107957326586
call (array([854]),)
279.1017560028175
call (array([855]),)
279.1025456985372
call (array([856]),)
279.1048781093371
call (array([857]),)
279.1041325485017
call (array([858]),)
279.1090977221212
call (array([859]),)
279.1034148687071
call (array([860]),)
279.102185310935
call (array([861]),)
279.10232198451985
call (array([862]),)
279.10802091676754
call (array([863]),)
279.1038247416132
call (array([864]),)
279.1087188734006
call (

...

279.0978171667763
call (array([9199]),)
279.0978171667763
call (array([9200]),)
279.0978171667763
call (array([9201]),)
279.0978171667763
call (array([9202]),)
279.0978171667763
call (array([9203]),)
279.0978171667763
call (array([9204]),)
279.0978171667763
call (array([9205]),)
279.0978171667763
call (array([9206]),)
279.0978171667763
call (array([9207]),)
279.0978171667763
call (array([9208]),)
279.0978171667763
call (array([9209]),)
279.0978171667763
call (array([9210]),)
279.0978171667763
call (array([9211]),)
279.0978171667763
call (array([9212]),)
279.0978171667763
call (array([9213]),)
279.0978171667763
call (array([9214]),)
279.1100430782885
call (array([9215]),)
279.1426505847016
call (array([9216]),)
279.1571551231796
call (array([9217]),)
279.19684921391075
call (array([9218]),)
279.2160463603452
call (array([9219]),)
279.2145480338791
call (array([9220]),)
279.2131830370315
call (array([9221]),)
279.21666010278926
call (array([9222]),)
279.22574748895573
call (array([9223])

279.21584345080413
call (array([9403]),)
279.2162265981601
call (array([9404]),)
279.21570526950495
call (array([9405]),)
279.2080677146771
call (array([9406]),)
279.2330112930382
call (array([9407]),)
279.2116563358063
call (array([9408]),)
279.2088135694879
call (array([9409]),)
279.2330723070936
call (array([9410]),)
279.23236083429316
call (array([9411]),)
279.2165040241157
call (array([9412]),)
279.21716562180154
call (array([9413]),)
279.2099785161224
call (array([9414]),)
279.2120024927114
call (array([9415]),)
279.21656400674016
call (array([9416]),)
279.204580401822
call (array([9417]),)
279.1943812254441
call (array([9418]),)
279.1900650379517
call (array([9419]),)
279.193534365295
call (array([9420]),)
279.15325796494307
call (array([9421]),)
279.13874286221426
call (array([9422]),)
279.1146917508562
call (array([9423]),)
279.1000843965293
call (array([9424]),)
279.0978171667763
call (array([9425]),)
279.0978171667763
call (array([9426]),)
279.0978171667763
call (array([9427

279.0978171667763
call (array([9607]),)
279.0978171667763
call (array([9608]),)
279.0978171667763
call (array([9609]),)
279.0978171667763
call (array([9610]),)
279.0978171667763
call (array([9611]),)
279.0978171667763
call (array([9612]),)
279.0978171667763
call (array([9613]),)
279.0978171667763
call (array([9614]),)
279.0978171667763
call (array([9615]),)
279.0978171667763
call (array([9616]),)
279.0978171667763
call (array([9617]),)
279.0978171667763
call (array([9618]),)
279.0978171667763
call (array([9619]),)
279.0978171667763
call (array([9620]),)
279.0978171667763
call (array([9621]),)
279.12519200256224
call (array([9622]),)
279.12955983537034
call (array([9623]),)
279.1602005610541
call (array([9624]),)
279.180983098997
call (array([9625]),)
279.18273758509514
call (array([9626]),)
279.20740096388954
call (array([9627]),)
279.1925272741327
call (array([9628]),)
279.2005381988862
call (array([9629]),)
279.1892873625582
call (array([9630]),)
279.1956142078086
call (array([9631])

279.16032276028125
call (array([9811]),)
279.1570264429697
call (array([9812]),)
279.1394058278581
call (array([9813]),)
279.1153746162938
call (array([9814]),)
279.1055923500082
call (array([9815]),)
279.1002106471344
call (array([9816]),)
279.0978171667763
call (array([9817]),)
279.0978171667763
call (array([9818]),)
279.0978171667763
call (array([9819]),)
279.0978171667763
call (array([9820]),)
279.0978171667763
call (array([9821]),)
279.0978171667763
call (array([9822]),)
279.0978171667763
call (array([9823]),)
279.0978171667763
call (array([9824]),)
279.0978171667763
call (array([9825]),)
279.0978171667763
call (array([9826]),)
279.0978171667763
call (array([9827]),)
279.0978171667763
call (array([9828]),)
279.0978171667763
call (array([9829]),)
279.0978171667763
call (array([9830]),)
279.0978171667763
call (array([9831]),)
279.0978171667763
call (array([9832]),)
279.0978171667763
call (array([9833]),)
279.0978171667763
call (array([9834]),)
279.0978171667763
call (array([9835]),)

 final_simplex: (array([[1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ],
       [1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ],
       [1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ],
       [1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ],
       [1740.72924805, 1740.72924805, 1740.72924805, ...,    0.        ,
           0.        ,    0.        ]]), array([279.062232  , 279.06728812, 279.07442241, ..., 279.255585  ,
       279.2562523 , 293.18340871]))
           fun: 279.0622319951583
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 10001
           nit: 1
        status: 2
       success: False
            

In [161]:
a = np.random.normal(100, 1, size=100)
b = np.random.normal(110, 1, size=100)
c = np.random.normal(120, 1, size=100)
d = np.random.normal(130, 1, size=100)
e = np.concatenate([a, b, c, d])


def e_rms(all_coefs):
    all_coefs = np.asarray(all_coefs, dtype=np.float64)
    if np.any(all_coefs <= 0): return 1000
    data = np.array([a, b, c, d]) * np.append([1], all_coefs)[:, np.newaxis]
    std  = data.flatten().std()
    return std

p0 = (1, 1, 1)
res = opt.minimize(e_rms, p0, tol=1e-8)
print(res)

      fun: 0.8489934690790586
 hess_inv: array([[0.00056088, 0.00025704, 0.00023723],
       [0.00025704, 0.00047184, 0.00021748],
       [0.00023723, 0.00021748, 0.0004014 ]])
      jac: array([7.4505806e-09, 7.4505806e-09, 7.4505806e-09])
  message: 'Optimization terminated successfully.'
     nfev: 95
      nit: 12
     njev: 19
   status: 0
  success: True
        x: array([0.90802675, 0.832879  , 0.76824317])


In [157]:
e.std()

11.206334849705366

In [162]:
100/res.x

array([110.12891467, 120.06545948, 130.16711827])